# CIP as a Service Data Challenge Demo

This Jupyter notebook will:
- Pull information about the state and tribe codes available from a Domains API
- Provide widgets to enter selections for the Indexing API
- Call the Indexing API
- Map the results

In [1]:
# First, some imports
import requests
import json
import ipywidgets as widgets
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl
import geopandas

In [4]:
# Call the Domains API
base_url = "https://cip-api.dmap-stage.aws.epa.gov/rpc/"
req_domains = requests.get(base_url + "cipsrv_domains", verify = False)
req_domains

C:\Users\EKeith\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cip-api.dmap-stage.aws.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [10]:
# Look at a sample of the data in the response
check = req_domains.json()
for key, value in check.items():
    if isinstance(value, list) and len(value) > 3:
        check[key] = value[:3]

check

{'states': [{'name': 'Alaska', 'geoid': '02', 'stusps': 'AK'},
  {'name': 'Alabama', 'geoid': '01', 'stusps': 'AL'},
  {'name': 'Arkansas', 'geoid': '05', 'stusps': 'AR'}],
 'tribes': [{'has_trust_lands': True,
   'aiannha_aiannhns': '01934337',
   'aiannha_namelsad': 'Acoma Pueblo',
   'aiannha_geoid_stem': '0010',
   'has_reservation_lands': True},
  {'has_trust_lands': True,
   'aiannha_aiannhns': '01934324',
   'aiannha_namelsad': 'Agua Caliente Indian Reservation',
   'aiannha_geoid_stem': '0020',
   'has_reservation_lands': True},
  {'has_trust_lands': True,
   'aiannha_aiannhns': '01350923',
   'aiannha_namelsad': 'Alabama-Coushatta Reservation',
   'aiannha_geoid_stem': '0050',
   'has_reservation_lands': True}]}

In [23]:
### Defines but does not yet display a map
map_input = Map(basemap=basemap_to_tiles(basemaps.OpenTopoMap), center=(38.8, -77.0), zoom=12)

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.6
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Please correct"
    },
    "allowIntersection": False
}
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Save the GeoJSON when it's drawn on the map"""
    feature_collection['features'].append(geo_json)
    
draw_control.on_draw(handle_draw)

map_input.add_control(draw_control)

In [22]:
# Define widgets needed to collect data for API
filter_for_widget = widgets.ToggleButtons(
    description = "Filter for", 
    options = [("States", "s"), ("Tribes", "t")]
)
nhd_version_widget = widgets.ToggleButtons(
    options = [("Medium Res (v2.1)", "nhdplus_m"), ("High Res", "nhdplus_h")],
    description = "NHDPlus Version"
)
state_clip_widget = widgets.Dropdown(
    options=[(st["name"], "USPS:" + st["stusps"]) 
             for st in req_domains.json()["states"]],
    description='State to clip',
    style=dict(description_width='initial'),
    layout=dict(visibility="visible")
)
tribe_clip_widget = widgets.Dropdown(
    options=[(tr["aiannha_namelsad"], "AIANNHNS:" + tr["aiannha_geoid_stem"]) 
             for tr in req_domains.json()["tribes"]],
    description='Tribe to clip',
    style=dict(description_width='initial'),
    layout=dict(visibility="hidden")
)
tribe_clip_type_widget = widgets.ToggleButtons(
    options=["All", "Tribe", "Reservation"],
    description='Clip tribe for',
    style=dict(description_width='initial'),
    layout=dict(visibility="hidden")
)
clip_when_widget = widgets.ToggleButtons(
    options = [("Before", "BEFORE"), ("After", "AFTER")],
    description = "When to clip"
)
catchment_filter_widget = widgets.Dropdown(
    options = [(st["name"], st["stusps"]) for st in req_domains.json()["states"]],
    description = "Catchment filter",
    style=dict(description_width='initial')
)
default_point_indexing_method_widget = widgets.ToggleButtons(
    options = [("Simple", "point_simple")],
    description = "Default point indexing method",
    value = "point_simple"
)
default_line_indexing_method_widget = widgets.ToggleButtons(
    options = [("Simple", "line_simple"), ("Level Path", "line_levelpath")],
    description = "Default line indexing method",
    value = "line_levelpath"
)
default_ring_indexing_method_widget = widgets.ToggleButtons(
    options = [("Simple", "area_simple"), ("Centroid", "area_centroid"), 
               ("Artificial Path", "area_artpath"), ("Treat rings as lines", "treat_as_lines")],
    description = "Default ring indexing method",
    value = "area_simple"
)
default_area_indexing_method_widget = widgets.ToggleButtons(
    options = [("Simple", "area_simple"), ("Centroid", "area_centroid"), 
               ("Artificial Path", "area_artpath")],
    description = "Default area indexing method",
    value = "area_simple"
)
default_line_threshold_widget = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    description='Line threshold',
    style=dict(description_width='initial')
)
default_areacat_threshold_widget = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    description='Area Catchment threshold',
    style=dict(description_width='initial')
)
default_areaevt_threshold_widget = widgets.IntSlider(
    value=1,
    min=0,
    max=100,
    description='Area Catchment threshold',
    style=dict(description_width='initial')
)
return_indexed_features_widget = widgets.ToggleButton(
    value=True,
    description='Return indexed features'
)
return_indexed_collection_widget = widgets.ToggleButton(
    value=False,
    description='Return indexed collection'
)
return_catchment_geometry_widget = widgets.ToggleButton(
    value=True,
    description='Return catchment geometry'
)
return_flowlines_widget = widgets.ToggleButton(
    value=True,
    description='Return flowlines'
)
return_flowline_geometry_widget = widgets.ToggleButton(
    value=True,
    description='Return flowline geometry'
)
return_huc12s_widget = widgets.ToggleButton(
    value=False,
    description='Return HUC12s'
)
# "wbd_version":null

def switch_state_tribe(change):
    if change.new == "s":
        state_clip_widget.layout.visibility = "visible"
        tribe_clip_widget.layout.visibility = "hidden"
        tribe_clip_type_widget.layout.visibility = "hidden"
    elif change.new == "t":
        state_clip_widget.layout.visibility = "hidden"
        tribe_clip_widget.layout.visibility = "visible"
        tribe_clip_type_widget.layout.visibility = "visible"
    else:
        state_clip_widget.layout.visibility = "hidden"
        tribe_clip_widget.layout.visibility = "hidden"
        tribe_clip_type_widget.layout.visibility = "hidden"

filter_for_widget.observe(switch_state_tribe, names = "value")

In [24]:
tab = widgets.Tab()
tab_clip = widgets.VBox([nhd_version_widget, filter_for_widget, state_clip_widget, tribe_clip_widget, tribe_clip_type_widget, clip_when_widget])
tab_indexing = widgets.VBox([default_point_indexing_method_widget, default_line_indexing_method_widget, default_ring_indexing_method_widget,default_area_indexing_method_widget, default_line_threshold_widget, default_areacat_threshold_widget, default_areaevt_threshold_widget])
tab_return = widgets.VBox([return_indexed_features_widget, return_indexed_collection_widget, return_catchment_geometry_widget, return_flowlines_widget, return_flowline_geometry_widget, return_huc12s_widget])
tab.children = [map_input, tab_clip, tab_indexing, tab_return]
tab.titles = ["Map", "Clipping", "Indexing", "What to Return"]
tab

In [25]:
# Convert widget values to data needed to call API
input_data={}
input_data["geometry"]=feature_collection
input_data["nhdplus_version"] = nhd_version_widget.value
if filter_for_widget.value == "s":
    input_data["geometry_clip"] = state_clip_widget.value
    input_data["catchment_filter"] = state_clip_widget.value
else:
    input_data["catchment_filter"] = "Tribe"
    if tribe_clip_type_widget == "All":
        input_data["geometry_clip"] = tribe_clip_widget.value
    elif tribe_clip_type_widget.value == "Tribe":
        input_data["geometry_clip"] = tribe_clip_widget.value + ":T"
    else:
        input_data["geometry_clip"] = tribe_clip_widget.value + ":R"
input_data["geometry_clip_stage"] = clip_when_widget.value
input_data["default_point_indexing_method"] = default_point_indexing_method_widget.value
input_data["default_line_indexing_method"] = default_line_indexing_method_widget.value
input_data["default_ring_indexing_method"] = default_ring_indexing_method_widget.value
input_data["default_area_indexing_method"] = default_area_indexing_method_widget.value
input_data["default_line_threshold"] = default_line_threshold_widget.value
input_data["default_areacat_threshold"] = default_areacat_threshold_widget.value
input_data["default_areaevt_threshold"] = default_areaevt_threshold_widget.value
input_data["return_indexed_features"] = return_indexed_features_widget.value
input_data["return_indexed_collection"] = return_indexed_collection_widget.value
input_data["return_catchment_geometry"] = return_catchment_geometry_widget.value
input_data["return_flowlines"] = return_flowlines_widget.value
input_data["return_flowline_geometry"] = return_flowline_geometry_widget.value
input_data["return_huc12s"] = return_huc12s_widget.value

In [26]:
req = requests.post(base_url + "cipsrv_index", data = json.dumps(input_data), verify = False)
req

C:\Users\EKeith\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cip-api.dmap-stage.aws.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [27]:
# Look at a sample of the data in the response
check = req.json()
for key, value in check.items():
    if isinstance(value, list) and len(value) > 3:
        check[key] = value[:3]

check

{'huc12s': None,
 'flowlines': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'LineString',
     'coordinates': [[-76.309573551, 37.76931072, 0],
      [-76.288078519, 37.770420154, 0],
      [-76.265296268, 37.733216227, 0],
      [-76.258492467, 37.7112284, 0]]},
    'properties': {'gnis_id': None,
     'fmeasure': 37.1753,
     'tmeasure': 100.0,
     'gnis_name': None,
     'nhdplusid': 10000300147503.0,
     'reachcode': '02080102104531'}}]},
 'catchments': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': None},
    'properties': {'areasqkm': 66.71189996,
     'nhdplusid': 10000300147503,
     'xwalk_huc12': '020801010000',
     'catchmentstatecode': 'VA'}}]},
 'return_code': 0,
 'indexed_areas': None,
 'indexed_lines': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'LineString',
     'coordinates': [[-76.24248, 37.883032],
      [-76.313896, 37.757194],
 

In [28]:
gdf = geopandas.GeoDataFrame.from_features(req.json()['flowlines']['features'], crs="EPSG:4326")
gdf

,geometry,gnis_id,fmeasure,tmeasure,gnis_name,nhdplusid,reachcode
0,"LINESTRING Z (-76.30957 37.76931 0.00000, -76....",None,37.1753,100.0,None,1.000030e+13,02080102104531


In [29]:
gdf.explore()